# Notebook for running the LoRA script (i.e. Stable Diffusion Fine-Tuning)

### Set the training parameters:

In [ ]:
training_params = {
    "hf_stablediffusion_name": "stabilityai/stable-diffusion-2", # set the name of the hugging face Stable Diffusion model
    "num_train_samples": 9025, # set the number of training samples (e.g., 9025 is the entire Imagewoof)
    "hf_dataset_name": "frgfm/imagewoof", # set the name of the hugging face dataset
    "hf_image_column": "image", # set the name of the image column in the hugging face dataset
    "prompt_type": "className_and_blip2", # set the prompt type to use (options: "only_className", "className_and_claude", "only_blip2", "className_and_blip2")
    "prompts_file": "../storage/prompts/imagewoof_blip2_finetuning.txt", # set the path to the prompts file
    "resolution": 224, # set the resolution of the images
    "train_batch_size": 1, # set the batch size
    "gradient_accumulation_steps": 4, # set the gradient accumulation steps
    "num_train_epochs": 3, # set the number of training epochs
    "learning_rate": 1e-4, # set the learning rate
    "mixed_precision": "fp16", # set the mixed precision
    "rank": 4, # set the LoRA rank
    "seed": 420, # set the random seed
    "output_dir": "../storage/finetuned_SD/imagewoof/sd_2/only_unet/className_and_blip2/1.00" # set the output directory for the finetuned model
}

### Run!

In [ ]:
!accelerate config default

In [ ]:
import time
import os

# Record the start time
start_time = time.time()

# Run the finetuning script
!accelerate launch lora.py \
    --pretrained_model_name_or_path={training_params["hf_stablediffusion_name"]} \
    --dataset_name={training_params["hf_dataset_name"]} --max_train_samples={training_params["num_train_samples"]} --image_column={training_params["hf_image_column"]} \
    --caption_column={training_params["prompt_type"]} --captions_file={training_params["prompts_file"]} \
    --resolution={training_params["resolution"]} --center_crop --random_flip \
    --train_batch_size={training_params["train_batch_size"]} --gradient_accumulation_steps={training_params["gradient_accumulation_steps"]} \
    --num_train_epochs={training_params["num_train_epochs"]} \
    --learning_rate={training_params["learning_rate"]} \
    --mixed_precision={training_params["mixed_precision"]} \
    --rank={training_params["rank"]} \
    --seed={training_params["seed"]} \
    --output_dir={training_params["output_dir"]} 

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
hours, rem = divmod(elapsed_time, 3600)
minutes, seconds = divmod(rem, 60)

# Write the elapsed time and training parameters to the log file
log_file_path = os.path.join(training_params["output_dir"], "finetuning_parameters.log")
with open(log_file_path, "w") as log_file:
    log_file.write("Finetuning Parameters:\n")
    for key, value in training_params.items():
        log_file.write(f"{key}: {value}\n")
    log_file.write(f"\nFinetuning Execution Time: {int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds\n")